In [32]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder



czyszczenie modelu z kolumn o dużych brakach - jeden z celów EDA. WYczytałem taką zasadę gdzieś na Medium. że jeżeli kolumna ma powyżej 30% braków względem wszystkich danych to można ją wyrzucić - wprowadza dużo szumu w danych. Jednak końcowo ważna jest analiza, które z tych kolumn mogą mieć wartość predykcyjną.

In [31]:

class Predykcja:
    def __init__(self,filename):
        self.model = None
        self.df = pd.read_csv(filename) 
        self.X, self.y, self.X_train, self.X_test, self.y_train, self.y_test = None,None,None,None, None, None
    def EDA(self):
        print(f"Informacje o danych {self.df.info()}")
        print(f"Shape {self.df.shape}")
        print(f"Liczba braków w danych {self.df.isnull().sum()}")
    def kolumny_do_usuniecia(self):
        ilosc_wszystkich_danych = self.df.shape[0]
        braki = self.df.isnull().sum()
        procent_brakow = (braki/ilosc_wszystkich_danych)*100
        do_usuniecia = pd.DataFrame({
            'Liczba brakow':braki,
            'procent': procent_brakow
        }).sort_values(by='procent',ascending=False)
        do_usuniecia['Do Usuniecia'] = do_usuniecia['procent']>30
        do_usuniecia_kolumny = do_usuniecia[do_usuniecia['Do Usuniecia']==True].index.tolist()
        print(f"Kolumny do usuniecia {do_usuniecia_kolumny}")
    def przygotowanie_danych(self):
        usuwanie = [['salary_range','required_education', 'benefits', 'required_experience', 'function']]
        self.df.drop(columns=usuwanie, inplace=True, errors='ignore')
        self.df['required_experience'].fillna('Not Specified', inplace=True)
        self.df['department'].fillna('Unknown', inplace=True)
        for col in self.df.select_dtypes(include='object').columns:
            if col != 'fraudulent':  
                self.df[col] = self.df[col].astype(str) 
                le = LabelEncoder()
                self.df[col] = le.fit_transform(self.df[col])
        print(f"Dane zostały przygotowane")
    def dane(self):
        self.X = self.df.drop(["fraudulent"],axis=1)
        self.y = self.df["fraudulent"]
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42)
    def wybierz_najlepszy_model(self):
        modele = {
             "XGBoost": XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, n_jobs=-1, eval_metric='mlogloss'),
            "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
            "Gradient Boosting": GradientBoostingClassifier(n_estimators=50, learning_rate=0.1, max_depth=3, random_state=42),
            "Neural Network": MLPClassifier(hidden_layer_sizes=100, max_iter=500, random_state=42)
        }
    
        najlepszy_model = None
        najlepsza_dokladnosc = 0
        for model_nazwa, model in modele.items():
            model.fit(self.X_train, self.y_train)
            accuracy = model.score(self.X_test, self.y_test)
            print(f"Model {model_nazwa}: {accuracy: .2f}")
            if accuracy > najlepsza_dokladnosc:
                najlepsza_dokladnosc = accuracy
                najlepszy_model = model
        self.model = najlepszy_model
        
        
        
    def trenowanie(self):
        self.model.fit(self.X_train, self.y_train)
    def predict(self):
        y_pred=self.model.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, y_pred)
        print(f"Dokładnosc modelu to : {accuracy:.2f}")
        scores = cross_val_score(self.model, self.X_train, self.y_train, cv=5, scoring='accuracy')
        print(f"Średnia dokładnosc: {np.mean(scores):.2f}")


model = Predykcja("DataSets/fake_job_postings.csv")
model.EDA()
model.kolumny_do_usuniecia()
model.przygotowanie_danych()
model.dane()
model.wybierz_najlepszy_model()
model.trenowanie()
model.predict()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15185 non-null  object
 8   benefits             10670 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func